In [4]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [5]:
# import the path
import os
path = os.path.abspath(os.path.join("..", "08_portfolio_management", "portfolio_example.xlsx"))

# read the Excel file
df = pd.read_excel(path, sheet_name=None)

# show the first sheet's data
first_sheet_name = list(df.keys())[0]

# dict to dataframe
df = pd.DataFrame(df[first_sheet_name])

df

,Ticker,Last Closing Price,Somma di Q.ty
0,AAPL,232.5600,18.0000
1,AMD,168.5800,18.0000
2,ASML,654.3000,6.0000
3,BNB-USD,857.4000,13.0900
4,BR50.MI,21.8550,213.0000
5,BTC-USD,111968.6300,2.1571
6,DOGE-USD,0.2153,40270.0000
7,EQQQ.DE,489.6000,39.0000
8,ETH-USD,4459.3400,4.3100
9,EXXT.DE,194.5000,10.1148


In [6]:
# add here Last Closing Price data download from yf or tv
# for tv, need to set the correct exchange/market

"""
Looks like Yahoo change their API regularly.
I have to use fast_info instead of info and then there's a key called 'lastPrice':

CurrentPrice = Commod.fast_info['lastPrice']
"""
# import yfinance as yf
# from tvDatafeed import TvDatafeed, Interval
# tv = TvDatafeed()

"\nLooks like Yahoo change their API regularly.\nI have to use fast_info instead of info and then there's a key called 'lastPrice':\n\nCurrentPrice = Commod.fast_info['lastPrice']\n"

In [7]:
# rename columns for clarity
df.rename(columns={'Last Closing Price': 'Last_Price', 'Somma di Q.ty': 'Quantity'}, inplace=True)

# calculate portfolio value for each position
df['Position_Value'] = df['Last_Price'] * df['Quantity']

# display the portfolio
print("Portfolio Overview:")
df.head()

Portfolio Overview:


,Ticker,Last_Price,Quantity,Position_Value
0,AAPL,232.560,18.00,4186.080
1,AMD,168.580,18.00,3034.440
2,ASML,654.300,6.00,3925.800
3,BNB-USD,857.400,13.09,11223.366
4,BR50.MI,21.855,213.00,4655.115


In [8]:
print(f"\nTotal Portfolio Value: ${df['Position_Value'].sum():,.2f}")
print(f"Number of positions: {len(df)}")


Total Portfolio Value: $422,389.17
Number of positions: 24


In [9]:
# portfolio pct composition
df['Position_Weight'] = (df['Position_Value'] / df['Position_Value'].sum())*100
df.head()

,Ticker,Last_Price,Quantity,Position_Value,Position_Weight
0,AAPL,232.560,18.00,4186.080,0.991048
1,AMD,168.580,18.00,3034.440,0.718399
2,ASML,654.300,6.00,3925.800,0.929427
3,BNB-USD,857.400,13.09,11223.366,2.657115
4,BR50.MI,21.855,213.00,4655.115,1.102091


In [10]:
# # download df as csv
# df.to_csv("portfolio_data.csv", index=False)

In [11]:
# top 10 holdings
top_10 = df.nlargest(10, 'Position_Value')
top_10

,Ticker,Last_Price,Quantity,Position_Value,Position_Weight
5,BTC-USD,111968.6300,2.1571,241527.531773,57.181279
16,META,738.7000,60.0000,44322.000000,10.493167
8,ETH-USD,4459.3400,4.3100,19219.755400,4.550248
7,EQQQ.DE,489.6000,39.0000,19094.400000,4.520570
15,MC.PA,513.1000,22.0000,11288.200000,2.672464
3,BNB-USD,857.4000,13.0900,11223.366000,2.657115
12,IWDE.MI,101.5700,96.0000,9750.720000,2.308468
6,DOGE-USD,0.2153,40270.0000,8670.131000,2.052640
18,NKE,77.9200,79.0000,6155.680000,1.457348
22,TSLA,345.9800,16.0000,5535.680000,1.310564


In [12]:
# Position_Weight of top 10
top_10_weight = top_10['Position_Weight'].sum()
print(f"\nTop 10 Holdings Weight: {top_10_weight:.2f}%")


Top 10 Holdings Weight: 89.20%


### Portfolio Value Update

In [14]:
# # Last_Price data download from yfinance
# import yfinance as yf
# import datetime

# def download_last_price_individual(tickers, start_date, end_date):
#     """
#     Download last prices for each ticker individually to handle failures gracefully
#     """
#     last_prices = {}
    
#     for ticker in tickers:
#         try:
#             print(f"Downloading data for {ticker}...")
#             # Download data for individual ticker
#             data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            
#             if not data.empty and 'Close' in data.columns:
#                 # Get the most recent price and ensure it's a scalar value
#                 last_price = float(data['Close'].iloc[-1])
#                 last_prices[ticker] = last_price
#                 print(f"✓ {ticker}: ${last_price:.2f}")
#             else:
#                 print(f"✗ {ticker}: No data available")
#                 last_prices[ticker] = None
                
#         except Exception as e:
#             print(f"✗ {ticker}: Error - {str(e)}")
#             last_prices[ticker] = None
    
#     return last_prices

# tickers = df['Ticker'].tolist()

# end_date = datetime.datetime.now()
# start_date = end_date - datetime.timedelta(days=10)

# print(f"Downloading last prices for {len(tickers)} tickers...")
# print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
# print("-" * 50)

# last_prices = download_last_price_individual(tickers, start_date, end_date)

# # update Last_Price in df
# print("\n" + "-" * 50)
# print("Updating portfolio with new prices...")

# # Update prices, keeping original price if download failed
# for ticker in df['Ticker']:
#     if ticker in last_prices and last_prices[ticker] is not None:
#         df.loc[df['Ticker'] == ticker, 'Last_Price'] = last_prices[ticker]
#     else:
#         print(f"⚠️  Keeping original price for {ticker} (download failed)")

# # recalculate Position_Value and Position_Weight
# df['Position_Value'] = df['Last_Price'] * df['Quantity']

# df['Position_Weight'] = (df['Position_Value'] / df['Position_Value'].sum())*100

# # display updated portfolio
# print("\nUpdated Portfolio Overview:")
# print(df[['Ticker', 'Last_Price', 'Position_Value', 'Position_Weight']])

# # total portfolio value
# total_value = df['Position_Value'].sum()
# print(f"\nTotal Portfolio Value: ${total_value:,.2f}")

# # number of positions
# num_positions = len(df)
# print(f"Number of positions: {num_positions}")

# # Show which tickers were successfully updated
# successful_updates = [ticker for ticker, price in last_prices.items() if price is not None]
# failed_updates = [ticker for ticker, price in last_prices.items() if price is None]

# print(f"\nSuccessfully updated: {len(successful_updates)} tickers")
# if successful_updates:
#     print(f"✓ Updated: {', '.join(successful_updates)}")

# if failed_updates:
#     print(f"\n⚠️  Failed to update: {len(failed_updates)} tickers")
#     print(f"✗ Failed: {', '.join(failed_updates)}")

# # top 10 holdings
# top_10 = df.nlargest(10, 'Position_Value')
# print("\nTop 10 Holdings:")
# print(top_10[['Ticker', 'Position_Value', 'Position_Weight']])

# top_10_weight = top_10['Position_Weight'].sum()
# print(f"\nTop 10 Holdings Weight: {top_10_weight:.2f}%")

# # save updated portfolio to csv
# df.to_csv("portfolio_data.csv", index=False)
# print(f"\nPortfolio data saved to 'portfolio_data.csv'")